**PyTorch `torch.nn`**

The `torch.nn` module in PyTorch is a core library that provides a wide array of classes and functions designed to help developers build neural networks efficiently and effectively. It abstracts the complexity of creating and training neural networks by offering pre-built layers, loss functions, activation functions, and other utilities—enabling you to focus on designing and experimenting with model architectures.


---

### **Key Components of `torch.nn`**

#### **1. Modules (Layers)**

* **`nn.Module`**:
  The base class for all neural network modules. Your custom models and layers should subclass this class.

* **Common Layers**:
  Includes layers such as:

  * `nn.Linear` (fully connected layer)
  * `nn.Conv2d` (convolutional layer)
  * `nn.LSTM` (recurrent layer), and many others.

#### **2. Activation Functions**

Functions like:

* `nn.ReLU`
* `nn.Sigmoid`
* `nn.Tanh`

These introduce non-linearities to the model, allowing it to learn complex patterns.

#### **3. Loss Functions**

Provides standard loss functions to measure the difference between predicted and actual values, such as:

* `nn.CrossEntropyLoss`
* `nn.MSELoss`
* `nn.NLLLoss`


#### **4. Container Modules**

* **`nn.Sequential`**:
  A simple container to stack layers in order, making model definitions concise and readable.


#### **5. Regularization and Dropout**

* Layers like `nn.Dropout` and `nn.BatchNorm2d` help:

  * Prevent overfitting
  * Improve generalization to unseen data

---

## **Loading the Dataset and Preprocessing**

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')

df.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)

df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('diagnosis', axis=1), df['diagnosis'], test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [ ]:
X_train_tensor = X_train_tensor.float()
X_test_tensor = X_test_tensor.float()

## **Building the Model**

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Model(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.linear = nn.Linear(num_features, 1)   # Determine the input Shape and output shape
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)

        return out

In [ ]:
# Important Parameters
learning_rate = 0.1
epochs = 1

In [ ]:
model = Model(X_train_tensor.shape[1])

for epoch in range(epochs):
    y_pred = model(X_train_tensor)   # automatically calls the forward method

    #print(y_pred)

In [ ]:
print(model.linear.bias)
model.linear.weight

Parameter containing:
tensor([-0.1413], requires_grad=True)


Parameter containing:
tensor([[-0.0837, -0.0791,  0.1698, -0.1770,  0.0590, -0.1325,  0.1553, -0.0139,
          0.0943,  0.1167,  0.1377, -0.0431, -0.1400, -0.1686, -0.0667, -0.0493,
          0.0190, -0.0025, -0.0175,  0.1586, -0.1461, -0.0062,  0.0907, -0.0493,
         -0.1045, -0.0063, -0.0607, -0.0369, -0.1622, -0.0958]],
       requires_grad=True)

In [ ]:
!pip install torchinfo --quiet

In [ ]:
from torchinfo import summary

summary(model, input_size=(1, X_train_tensor.shape[1]))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 1]                    --
├─Linear: 1-1                            [1, 5]                    155
├─ReLU: 1-2                              [1, 5]                    --
├─Linear: 1-3                            [1, 1]                    6
├─Sigmoid: 1-4                           [1, 1]                    --
Total params: 161
Trainable params: 161
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

### Model with multiple neurons

In [ ]:
class Model(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.linear1 = nn.Linear(num_features, 5)   # Determine the input Shape and output shape
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(5, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)

        return out

In [ ]:
summary(model, input_size=(1, X_train_tensor.shape[1]))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [1, 1]                    --
├─Linear: 1-1                            [1, 5]                    155
├─ReLU: 1-2                              [1, 5]                    --
├─Linear: 1-3                            [1, 1]                    6
├─Sigmoid: 1-4                           [1, 1]                    --
Total params: 161
Trainable params: 161
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [ ]:
# Important Parameters
learning_rate = 0.1
epochs = 1

In [ ]:
model = Model(X_train_tensor.shape[1])

for epoch in range(epochs):
    y_pred = model(X_train_tensor)   # automatically calls the forward method

    #print(y_pred)

In [ ]:
print(model.linear1.bias)
model.linear1.weight

Parameter containing:
tensor([-0.1535,  0.0039,  0.1467,  0.0200,  0.1118], requires_grad=True)


Parameter containing:
tensor([[-0.1272,  0.0085,  0.0824, -0.0858,  0.1221,  0.0194, -0.0699,  0.1032,
         -0.0596,  0.0322,  0.0394,  0.1606, -0.0604, -0.1620, -0.0108, -0.1331,
         -0.0544,  0.0965, -0.1692,  0.0990,  0.0953,  0.1450,  0.1659, -0.1745,
          0.1037,  0.1253,  0.0013,  0.1438, -0.0170,  0.0251],
        [-0.1387,  0.1153, -0.0031,  0.1116,  0.0851, -0.1203,  0.1747, -0.1557,
         -0.0482,  0.1031, -0.1298, -0.0353,  0.1060, -0.0078, -0.0465, -0.0598,
         -0.1047, -0.1004,  0.0507,  0.1254, -0.0135, -0.0383, -0.1576,  0.1012,
         -0.0519,  0.0467, -0.1398,  0.0535, -0.1086, -0.0653],
        [ 0.1108, -0.1421,  0.0505, -0.0451, -0.1094, -0.0583,  0.1714,  0.1777,
         -0.0374, -0.1585,  0.1543,  0.1511,  0.1032,  0.0127,  0.0289, -0.0906,
          0.1399, -0.0571, -0.0387, -0.0624, -0.0535, -0.1128, -0.1499, -0.1606,
          0.0930, -0.0974, -0.0893, -0.1548, -0.1553,  0.0403],
        [-0.0859,  0.0559, -0.0404,  0.0196,  0.1683,  0.

In [ ]:
# create model class
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, num_features):

        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 5),
            nn.ReLU(),
            nn.Linear(5, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        out = self.network(x)

        return out


    def evaluate_model(self, x_test, y_test):
        with torch.no_grad():
            y_pred = self.forward(x_test)
            y_pred = (y_pred > 0.5).float()
            accuracy = (y_pred == y_test).float().mean()
            return accuracy

## Improved Code

---

#### **Understanding `torch.optim` in PyTorch**

The `torch.optim` module in PyTorch provides a collection of **optimization algorithms** used to update the parameters of a model during training. It includes commonly used optimizers such as:

* `SGD` (Stochastic Gradient Descent)
* `Adam`
* `RMSprop`
* And others

These optimizers efficiently handle weight updates and offer additional features like:

* **Learning rate scheduling**
* **Weight decay** (used for regularization)

---

### **`model.parameters()` in PyTorch**

The `model.parameters()` method returns an **iterator over all the trainable parameters** in a model. These parameters are instances of `torch.nn.Parameter` and typically include:

* **Weights**:
  The weight matrices of layers like `nn.Linear`, `nn.Conv2d`, etc.

* **Biases**:
  The bias terms of layers (if applicable)

These parameters are passed to the optimizer, which then:

1. **Computes gradients** via backpropagation
2. **Updates parameters** using the selected optimization algorithm

---


In [ ]:
# create model class
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, num_features):

        super().__init__()
        self.linear = nn.Linear(num_features, 1)   # Determine the input Shape and output shape
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)

        return out

    def evaluate_model(self, x_test, y_test):
        with torch.no_grad():
            y_pred = self.forward(x_test)
            y_pred = (y_pred > 0.5).float()
            accuracy = (y_pred == y_test).float().mean()
            return accuracy

In [ ]:
# Important Parameters
learning_rate = 0.1
epochs = 25

In [ ]:
loss_function = nn.BCELoss()

In [ ]:
model = Model(X_train_tensor.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # SDG : Stochastic Gradient

for epoch in range(epochs):
    # Forward Pass
    y_pred = model(X_train_tensor)   # automatically calls the forward method

    # Calculate Loss
    loss = loss_function(y_pred.flatten(), y_train_tensor.float())

    # Clear Gradients
    optimizer.zero_grad()

    # back prop
    loss.backward()

    # Update Parameters
    optimizer.step()

    acc = model.evaluate_model(X_test_tensor, y_test_tensor)

    print(f"Epoch: {epoch+1}, Loss : {loss.item():.4f}, Accuracy : {acc.item():.2f}")

Epoch: 1, Loss : 0.7483, Accuracy : 0.38
Epoch: 2, Loss : 0.7362, Accuracy : 0.38
Epoch: 3, Loss : 0.7244, Accuracy : 0.38
Epoch: 4, Loss : 0.7128, Accuracy : 0.38
Epoch: 5, Loss : 0.7014, Accuracy : 0.38
Epoch: 6, Loss : 0.6903, Accuracy : 0.38
Epoch: 7, Loss : 0.6794, Accuracy : 0.38
Epoch: 8, Loss : 0.6685, Accuracy : 0.38
Epoch: 9, Loss : 0.6578, Accuracy : 0.38
Epoch: 10, Loss : 0.6472, Accuracy : 0.38
Epoch: 11, Loss : 0.6368, Accuracy : 0.38
Epoch: 12, Loss : 0.6265, Accuracy : 0.38
Epoch: 13, Loss : 0.6162, Accuracy : 0.39
Epoch: 14, Loss : 0.6061, Accuracy : 0.41
Epoch: 15, Loss : 0.5961, Accuracy : 0.42
Epoch: 16, Loss : 0.5862, Accuracy : 0.47
Epoch: 17, Loss : 0.5764, Accuracy : 0.53
Epoch: 18, Loss : 0.5666, Accuracy : 0.53
Epoch: 19, Loss : 0.5571, Accuracy : 0.53
Epoch: 20, Loss : 0.5476, Accuracy : 0.53
Epoch: 21, Loss : 0.5382, Accuracy : 0.53
Epoch: 22, Loss : 0.5290, Accuracy : 0.53
Epoch: 23, Loss : 0.5198, Accuracy : 0.53
Epoch: 24, Loss : 0.5108, Accuracy : 0.53
E